In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import kneighbors_graph
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from networkx.algorithms.community import k_clique_communities, girvan_newman
from networkx import edge_betweenness_centrality as betweenness
import matplotlib.pyplot as plt 
%matplotlib inline

In [10]:
inputData = {
    'product/productId': [],
    'review/userId': [],
    'review/profileName': [],
    'review/helpfulness': [],
    'review/score': [],
    'review/time': [],
    'review/summary': [],
    'review/text': []
}

In [11]:
# Read data from the input dataset file and create a dictionary of input data with each column as key
input_file = open('sample_data.txt', 'r')
key_value = []
for line in input_file:
    if len(line.strip()) != 0:
        key_value = line.split(':')
        if key_value[0].strip() in inputData.keys():
            inputData[key_value[0].strip()].append(key_value[1].strip())
        else:
            continue

In [12]:
# Create dataframe from the input data dictionary and split the dataframe into train and test sets
train_df = pd.DataFrame.from_dict(inputData)
# train_df, test_df = train_test_split(df, test_size=0.2)
#print(df.shape, train_df.shape, test_df.shape)
train_df.head()

,product/productId,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,5.0,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,1.0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1/1,4.0,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3/3,2.0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0/0,5.0,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [13]:
train_df.describe()

,product/productId,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
count,939,939,939,939,939,939,939,939
unique,193,906,902,65,5,706,898,934
top,B000G6RYNE,A3PJZ8TU8FDQ1K,Jared Castle,0/0,5.0,1334016000,Delicious!,"I'm addicted to salty and tangy flavors, so wh..."
freq,217,5,5,450,600,5,8,3


In [14]:
# Create a dictionary from train data
# key -> product
# value -> concatenated reviews by various users for the product
# Finally extract list of unique produccts and concatenated reviews from the dictionary
product_reviews_dict = {}
for index, row in train_df.iterrows():
    if row['product/productId'] in product_reviews_dict:
        product_reviews_dict[row['product/productId']] = product_reviews_dict[row['product/productId']] + " " + row['review/text']
    else:
        product_reviews_dict[row['product/productId']] = row['review/text']
products_list = list(product_reviews_dict.keys())
reviews_list = list(product_reviews_dict.values())

In [15]:
# Tokenize the review text using CountVectorizer to create a document term matrix (product vs words in review)
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(reviews_list)

In [16]:
# Reduce the dimensionality of document term matrix
svd = TruncatedSVD(n_components=150, n_iter=5)
X_red = svd.fit_transform(X)
X_red.shape

(193, 150)

In [17]:
# Create an adjacency matrix for graph using kneighbors_graph with speciefied number of neighbors
X_graph_adj = kneighbors_graph(X_red, 10, mode='distance', n_jobs=-1)
# X_graph.toarray()

In [18]:
# Create a networkX graph from the adjacency matrix
main_graph = nx.from_scipy_sparse_matrix(X_graph_adj)

# Create a label mapping between products and indices created in the graph
label_mapping = dict(zip(main_graph.nodes(), products_list))

# Relabel the indices in the graph with product names 
main_graph = nx.relabel_nodes(main_graph, label_mapping)
print(nx.info(main_graph))

Name: 
Type: Graph
Number of nodes: 193
Number of edges: 1661
Average degree:  17.2124


In [19]:
# function to calculate pagerank for the nodes in a community within a graph
def getPageRankOfCommunity(G, community_nodes):
    community_graph = G.subgraph(community_nodes)
    return nx.pagerank(community_graph, alpha=0.85, weight='weight')


Idenntify Communities using K-Clique community detection algorithm

In [20]:
# Instanciate K-Clique algorithm and get the communities 
k_clique = k_clique_communities(main_graph, 6)

# Transform the communities obtained into a dictionary with
# key -> communityId
# value -> list of nodes in the community
kc_dict = dict(enumerate(k_clique))

# Create a dictionary with
# key -> product
# value -> Id of the community the product belongs to
kc_product_comm_dict = {}
for comm, products in kc_dict.items():
    for product in products:
        kc_product_comm_dict[product] = comm

# Calculate pagerank for the nodes in each community and create a dictionary where
# key -> communityId
# value -> list of tuples (product, pagerank) sorted in descending order of pagerank
kc_community_pagerank_dict = {}
for comm, nodes in kc_dict.items():
    page_rank_dict = getPageRankOfCommunity(main_graph, nodes)
    kc_community_pagerank_dict[comm] = sorted(page_rank_dict.items(), key=lambda kv: kv[1], reverse=True)
# print(kc_community_pagerank_dict)

Identify Communities using Girvan-Newman community detection algorithm

In [21]:
# define a function to evaluate the most valuable edge considering the weights of the edges
# then use the function in girvan newman algorith to remove the edge returned byb this function in each iteration
def most_central_edge(G):
    centrality = betweenness(G, weight='weight')
    return max(centrality, key=centrality.get)
gn_comm = girvan_newman(main_graph, most_valuable_edge=most_central_edge)

# get the communities using girvan newman algorithm
first_iteration_comm = tuple(sorted(c) for c in next(gn_comm))

# Transform the communities obtained into a dictionary with
# key -> communityId
# value -> list of nodes in the community
gn_dict = dict(enumerate(first_iteration_comm))

# Create a dictionary with
# key -> product
# value -> Id of the community the product belongs to
gn_product_comm_dict = {}
for comm, products in gn_dict.items():
    for product in products:
        gn_product_comm_dict[product] = comm

# Calculate pagerank for the nodes in each community and create a dictionary where
# key -> communityId
# value -> list of tuples (product, pagerank) sorted in descending order of pagerank
gn_community_pagerank_dict = {}
for comm, nodes in gn_dict.items():
    page_rank_dict = getPageRankOfCommunity(main_graph, nodes)
    gn_community_pagerank_dict[comm] = sorted(page_rank_dict.items(), key=lambda kv: kv[1], reverse=True)
# print(gn_community_pagerank_dict)

In [22]:
# function to return the recommendation of a product using the communities specified
# product_comm_dict and community_pagerank_dict can be results of any community detection alorithm (K-clique or girvan-newman)
def getProductRecommendations(product, product_comm_dict, community_pagerank_dict):
    if product in product_comm_dict:
        recommendation_list = []
        comm_nodes = community_pagerank_dict[product_comm_dict[product]]
        comm_nodes = [(p, pr*cosine_similarity(X_red[list(products_list).index(p)].reshape(1,-1), X_red[list(products_list).index(product)].reshape(1,-1))) for p, pr in comm_nodes]
        comm_nodes = sorted(comm_nodes, key=lambda kv: kv[1], reverse=True)
        for product_id, pagerank in comm_nodes:
            if len(recommendation_list) >= 5:
                break
            elif product_id != product:
                recommendation_list.append(product_id)
            else:
                continue
        return recommendation_list
    else:
        return []

Predict recommendations using communities of K-Clique algorithm

In [24]:
train_df, test_df = train_test_split(train_df, test_size=0.02)
for product in test_df['product/productId']:
    print("Recommendations for '" + product + "' are: " + str(getProductRecommendations(product, kc_product_comm_dict, kc_community_pagerank_dict)))

Recommendations for 'B004ET7MG8' are: ['B0007NG568', 'B000ER6YO0', 'B000H13270', 'B003ZFRKGO', 'B000HDMUQ2']
Recommendations for 'B006K2ZZ7K' are: []
Recommendations for 'B0019CW0HE' are: ['B000ER6YO0', 'B003AO5DLO', 'B0009XLVGA', 'B000HKYP9A', 'B0007NG568']
Recommendations for 'B000G6RYNE' are: ['B000G6MBX2', 'B001D07IPG', 'B000H13270', 'B0007NG568', 'B000HDMUQ2']
Recommendations for 'B000UWSQT0' are: ['B00285FF6O', 'B004X2KR36', 'B000VKYKTG', 'B000IXUISS', 'B0093NIWVO']
Recommendations for 'B000G6RYNE' are: ['B000G6MBX2', 'B001D07IPG', 'B000H13270', 'B0007NG568', 'B000HDMUQ2']
Recommendations for 'B001LO4ZWI' are: ['B000H13270', 'B0007NG568', 'B000ER6YO0', 'B000HDMUQ2', 'B003ZFRKGO']
Recommendations for 'B000ER6YO0' are: ['B000H13270', 'B000G6MBX2', 'B0040WAG7Q', 'B0007NG568', 'B003OB0IB8']
Recommendations for 'B000G6RYNE' are: ['B000G6MBX2', 'B001D07IPG', 'B000H13270', 'B0007NG568', 'B000HDMUQ2']
Recommendations for 'B000G6RYNE' are: ['B000G6MBX2', 'B001D07IPG', 'B000H13270', 'B0007

Predict recommendations using communities of Girvan-Newman algorithm

In [25]:
for product in test_df['product/productId']:
    print("Recommendations for '" + product + "' are: " + str(getProductRecommendations(product, gn_product_comm_dict, gn_community_pagerank_dict)))

Recommendations for 'B004ET7MG8' are: ['B0007NG568', 'B000H13270', 'B000ER6YO0', 'B001EPPFGO', 'B001EO5ZMO']
Recommendations for 'B006K2ZZ7K' are: ['B001EO5ZMO', 'B000ER6YO0', 'B003SE19UK', 'B003OB0IB8', 'B004N5KULM']
Recommendations for 'B0019CW0HE' are: ['B000ER6YO0', 'B003SE19UK', 'B003AO5DLO', 'B0007NG568', 'B003ZFRKGO']
Recommendations for 'B000G6RYNE' are: ['B000H13270', 'B0007NG568', 'B000ER6YO0', 'B001EPPFGO', 'B001EO5ZMO']
Recommendations for 'B000UWSQT0' are: ['B000ER6YO0', 'B00285FF6O', 'B001ELL9X6', 'B000LKZK7C', 'B003ZFRKGO']
Recommendations for 'B000G6RYNE' are: ['B000H13270', 'B0007NG568', 'B000ER6YO0', 'B001EPPFGO', 'B001EO5ZMO']
Recommendations for 'B001LO4ZWI' are: ['B000H13270', 'B0025VRCJY', 'B0007NG568', 'B001EPPFGO', 'B000ER6YO0']
Recommendations for 'B000ER6YO0' are: ['B000H13270', 'B003ZFRKGO', 'B0007NG568', 'B001EO5ZMO', 'B000HKYP9A']
Recommendations for 'B000G6RYNE' are: ['B000H13270', 'B0007NG568', 'B000ER6YO0', 'B001EPPFGO', 'B001EO5ZMO']
Recommendations for